# libraries

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

Let's set device

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [37]:
bs = 100

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])


train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [38]:
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2)
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        return torch.tanh(self.fc4(x))

class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))

In [39]:
# build network
z_dim = 100
mnist_dim = train_dataset.train_data.size(1) * train_dataset.train_data.size(2)

G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim).to(device)
D = Discriminator(mnist_dim).to(device)

In [40]:
G

Generator(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=784, bias=True)
)

In [41]:
D

Discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)

In [42]:
# loss
criterion = nn.BCELoss()

# optimizer
lr = 0.0002
G_optimizer = optim.Adam(G.parameters(), lr = lr)
D_optimizer = optim.Adam(D.parameters(), lr = lr)

In [43]:
def D_train(x):
    #=======================Train the discriminator=======================#
    D.zero_grad()

    # train discriminator on real
    x_real, y_real = x.view(-1, mnist_dim), torch.ones(bs, 1)
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

    D_output = D(x_real)
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    # train discriminator on facke
    z = Variable(torch.randn(bs, z_dim).to(device))
    x_fake, y_fake = G(z), Variable(torch.zeros(bs, 1).to(device))

    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()

    return  D_loss.data.item()

In [44]:
def G_train(x):
    #=======================Train the generator=======================#
    G.zero_grad()

    z = Variable(torch.randn(bs, z_dim).to(device))
    y = Variable(torch.ones(bs, 1).to(device))

    G_output = G(z)
    D_output = D(G_output)
    G_loss = criterion(D_output, y)

    # gradient backprop & optimize ONLY G's parameters
    G_loss.backward()
    G_optimizer.step()

    return G_loss.data.item()

In [45]:
n_epoch = 100

test_z = Variable(torch.randn(bs, z_dim).to(device))
for epoch in range(1, n_epoch+1):
    D_losses, G_losses = [], []
    for batch_idx, (x, _) in enumerate(train_loader):
        D_losses.append(D_train(x))
        G_losses.append(G_train(x))

    with torch.no_grad():

        generated = G(test_z)
        save_image(generated.view(generated.size(0), 1, 28, 28), f'./samples/sample{epoch}.png')

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
        epoch, n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))


[1/100]: loss_d: 0.835, loss_g: 3.442
[2/100]: loss_d: 0.960, loss_g: 2.298
[3/100]: loss_d: 1.061, loss_g: 1.858
[4/100]: loss_d: 0.890, loss_g: 2.225
[5/100]: loss_d: 0.713, loss_g: 2.059
[6/100]: loss_d: 0.527, loss_g: 2.718
[7/100]: loss_d: 0.589, loss_g: 2.399
[8/100]: loss_d: 0.604, loss_g: 2.499
[9/100]: loss_d: 0.655, loss_g: 2.369
[10/100]: loss_d: 0.600, loss_g: 2.492
[11/100]: loss_d: 0.624, loss_g: 2.427
[12/100]: loss_d: 0.722, loss_g: 2.185
[13/100]: loss_d: 0.668, loss_g: 2.330
[14/100]: loss_d: 0.715, loss_g: 2.142
[15/100]: loss_d: 0.761, loss_g: 2.031
[16/100]: loss_d: 0.808, loss_g: 1.923
[17/100]: loss_d: 0.803, loss_g: 1.886
[18/100]: loss_d: 0.853, loss_g: 1.770
[19/100]: loss_d: 0.871, loss_g: 1.683
[20/100]: loss_d: 0.890, loss_g: 1.656
[21/100]: loss_d: 0.877, loss_g: 1.693
[22/100]: loss_d: 0.911, loss_g: 1.649
[23/100]: loss_d: 0.920, loss_g: 1.626
[24/100]: loss_d: 0.911, loss_g: 1.592
[25/100]: loss_d: 0.961, loss_g: 1.501
[26/100]: loss_d: 0.952, loss_g: 1

In [47]:
import imageio
import os

image_folder = "./samples"
gif_filename = "gan_training.gif"

images = sorted([img for img in os.listdir(image_folder) if img.endswith(".png")], key=lambda x: int(x[6:-4]))
imageio.mimsave(gif_filename, [imageio.imread(f"{image_folder}/{img}") for img in images], duration=0.2)


<ipython-input-47-05f239421fa4>:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imageio.mimsave(gif_filename, [imageio.imread(f"{image_folder}/{img}") for img in images], duration=0.2)


In [48]:
from moviepy.editor import VideoFileClip

def convert_gif_to_mp4(input_gif, output_mp4):
    clip = VideoFileClip(input_gif)
    clip.write_videofile(output_mp4, codec="libx264", fps=clip.fps)

# Example usage
convert_gif_to_mp4("gan_training.gif", "output.mp4")


  if event.key is 'enter':



Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
